# Tutorial 05: Discrete Counts Responses

#### Lecture and Tutorial Learning Goals:
After completing this week's lecture and tutorial work, you will be able to:

1. Describe the Poisson regression estimation procedure (discrete counts as the response variable and explanatory variables).
2. Interpret the coefficients and $p$-values in the Poisson regression settings.
3. Discuss useful Poisson regression diagnostics and explain why they should be performed.
4. Write a computer script to perform Poisson regression and perform model diagnostics. Interpret and communicate the results from that computer script.

In [ ]:
# Run this cell before continuing.
library(broom)
library(MASS)
library(glmbb)
library(cowplot)
library(tidyverse)
library(faraway)

source("tests_tutorial_05.R")

## Poisson Regression

- Poisson regression is a regression analysis designed for modelling **count data**, where the response variable represents the number of times an event occurs within a fixed interval of time or space. 

- Since **counts are nonnegative integers**, if we use linear regression, we might have a range problem by predicting negative counts. 

- By employing Poisson regression, we can analyze and predict event counts, such as the number of customer arrivals, disease occurrences, or traffic accidents, providing valuable insights into the factors influencing these events and enabling better decision-making.

**The Poisson Regression**

The Poisson regression model is another model in the family of *generalized linear models*, so you can use the function `glm()` to estimate it.

Since the expected value of counts is always positive, we can't use a linear regression to model it. Instead, we use a *function* of the linear component with a range $[0, + \inf)$ to model the conditional expectation

$$E[Y_i|\mathbf{X}_i] = e^{\beta_0 + \beta_1X_{i,1} + \ldots + \beta_pX_{i,p}}$$

or equivalently,

$$\log(E[Y_i|\mathbf{X}_i]) = \beta_0 + \beta_1X_{i,1} + \ldots + \beta_pX_{i,p}$$

> The function that links the conditional expectation to the linear component
>
> exponential function is not the only function that works but it is commonly used and the default link for this family

Usually, the mean of a Poisson distribution is called $\lambda$. This is an arbitrary choice, like using $\mu$ for the Normal distribution.

## Galapagos Islands

![img](https://naturegalapagos.com/wp-content/uploads/2013/10/where-are-the-galapagos-islands-map.jpg.webp)

In this tutorial you will work with the `galapagos` dataset, from the `faraway` package that contains information about 30 different Galapagos islands and the number of plant species found in each island. 

We will explore the relationship between the number of plant species in the islands and several geographic variables about the islands in this dataset. 

More information about this dataset can be found using `?gala`

In [ ]:
# Load the data
galapagos <- 
    gala %>% 
    as_tibble(rownames = 'island') %>%
    select(-Endemics)

colnames(galapagos) <- str_to_lower(colnames(galapagos))

galapagos %>% 
    slice_sample(n = 3)

Let's start by exploring the association between the number of plant species found in an island, `species,` and the highest elevation of the island `elevation`.

We want to estimate the following Poisson regression: 

$$
\log(\lambda_i) = \beta_0 + \beta_1\times\text{elevation}_i
$$

where $\lambda_i$ is the mean of the random variable `species`. 

> We assume that (conditional of any of these covariates) `species` follows a Poisson distribution.

**Question 1.1**
<br>{points: 1}

Create a scatterplot of `species` versus `elevation` (via `geom_point()`). The `ggplot()` object's name will be `species_elevation_plot`. Recall that the response must be placed on the $y$-axis, whereas the continuous input must be on the $x$-axis. Include proper axis labels and title.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# Adjust these numbers so the plot looks good in your desktop.
options(repr.plot.width = 7, repr.plot.height = 5) 

# species_elevation_plot <- 
#   ... %>%
#   ggplot() +
#   ...(aes(..., ...)) +
#   labs(y = "Number of Species", x = "Elevation (mts)") +
#   ggtitle(Scatterplot of Number of Species vs Elevation) +
#   theme(text = element_text(size = 14)) 

### BEGIN SOLUTION
species_elevation_plot <- 
  galapagos %>%
  ggplot() +
  geom_point(aes(elevation, species)) +
  labs(y = "Number of Species", x = "Elevation (mts)") +
  ggtitle("Scatterplot of Number of Species vs Elevation") +
  theme(text = element_text(size = 14))

### END SOLUTION

species_elevation_plot

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 1}

By looking at `species_elevation_plot`, graphically speaking, what is the relationship between `species` and carapace `elevation`?

**A.** Positive.

**B.** Negative.

**C.** No relationship.

*Assign your answer to the object `answer1.2` (character type surrounded by quotes).*

In [ ]:
# answer1.2 <- ...

### BEGIN SOLUTION
answer1.2 <- "A"
### END SOLUTION

In [ ]:
test_1.2()

**Question 1.3**
<br>{points: 1}

Let us plot the estimated model on top of `species_elevation_plot` using `geom_smooth()` with `method = "glm"` and `method.args = list(family = poisson)`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# Adjust these numbers so the plot looks good in your desktop.
options(repr.plot.width = 9, repr.plot.height = 5)

# species_elevation_plot <- 
#   species_elevation_plot +
#   ...(aes(..., ...), 
#       ...,
#       se = FALSE,
#       ...)

### BEGIN SOLUTION
species_elevation_plot <- 
    species_elevation_plot +
    geom_smooth(aes(elevation, species), 
                method = "glm", 
                se = FALSE,
                method.args = list(family = poisson))
### END SOLUTION

species_elevation_plot

In [ ]:
test_1.3()

### Estimation

As seen for other models, the parameters $\beta_0, \beta_1, \dots, \beta_{p}$ are unknown population coefficients that we want to estimate using data. 

In order to fit a Poisson regression model, we can use the function `glm()` and its argument `family = poisson` (required to specify the Poisson nature of the response).

> Poisson regression is another model of the Generalized Linear Model family

Estimates of the regression coefficients, $\hat{\beta}_0, \hat{\beta}_1, \dots \hat{\beta}_{p}$, are obtained through Maximum Likelihood.

**Question 1.4**
<br>{points: 1}

Using `glm()`, estimate a Poisson regression model with `species` as a response and `elevation` as a continuous covariate.

Call the estimated model `species_elevation_model`.
    
*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*    

In [ ]:
# species_elevation_model <- ...(...,
#                            ...,
#                            ...)

### BEGIN SOLUTION
species_elevation_model <- glm(formula = species ~ elevation,
                           data = galapagos,
                           family = poisson)
### END SOLUTION

summary(species_elevation_model)

In [ ]:
test_1.4()

**Question 1.5**
<br>{points: 1}

Report the estimated coefficients, their standard errors, and corresponding $p$-values using `tidy()` with `species_elevation_model`. Include the corresponding asymptotic 95% confidence intervals. 

> Note: do not exponentiate the coefficients in this first question

Store the results in the variable `species_elevation_model_results`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# species_elevation_model_results <- 
#    ...
#    ...(... = TRUE) %>%
#    mutate_if(is.numeric, round, 4)

### BEGIN SOLUTION
species_elevation_model_results <- 
    species_elevation_model %>%
    tidy(conf.int = TRUE) %>%
    mutate_if(is.numeric, round, 4)
### END SOLUTION

species_elevation_model_results

In [ ]:
test_1.5()

**Question 1.6**
<br>{points: 1}

As done for logistic regression, we can also interpret the exponentiated coefficients since we are using a logarithmic function to link the conditional expectation to the linear component. Use `exponentiate = TRUE` in `tidy()` to get the exponentiated estimates and the corresponding 95% confidence intervals. 

> Note these are the estimates of the coefficients in the model of the mean of the number of plant species (i.e., $E[\texttt{species}_i|\texttt{elevation}_i]$).

Round numeric results to 4 decimal places.

Call the resulting object `species_elevation_exp_results`. 

*This time without a skeleton, you've done this before!  Write your code in the cell below, and run it.*

In [ ]:
# write your own code here 

### BEGIN SOLUTION
species_elevation_exp_results <- 
    species_elevation_model %>%
    tidy(exponentiate = TRUE, conf.int = TRUE) %>%
    mutate_if(is.numeric, round, 4)
### END SOLUTION

species_elevation_exp_results


In [ ]:
test_1.6()

Note that `std.error` and `statistic` are not adjusted! Only the estimates and confidence limits are exponentiated.

**Question 1.7**
<br>{points: 1}

Provide an interpretation of the coefficient of `elevation` in the model of the mean of the number of plant species (i.e., $E[\texttt{species}_i|\texttt{elevation}_i]$). 

> recall, when observational data is used, you can only establish an association between variables

> *Your answer goes here.*

### BEGIN SOLUTION
An increase of 1 meter in the maximum elevation of an island is associated with a change in the mean (or average) number of plant species found by a factor of $e^{0.0014} = 1.00144$, i.e., an increase of $0.14\%$.

Rubric note: either interpretation is correct

(-.5): if they say "keeping all variables constant" since there are not other variables

(-.5): if they provide an interpretation in terms of the log mean

(-1): if the raw estimates are used with a multiplicative interpretation 
### END SOLUTION

**Question 1.8**
<br>{points: 1}

Using the output in `species_elevation_exp_results`, which of the following statements is correct?

**A.** With 95% confidence we expect an increment in the mean number of plant species between 14% and 15% per 1 meter increase in the highest elevation of an island.

**B.** With 95% confidence we expect an increment in the mean number of plant species between 0.14% and 0.15% per 1 meter increase in the highest elevation of an island.

**C.** With 95% probability we expect an increment in the mean number of plant species between 14% and 15% per 1 meter increase in the highest elevation of an island.

**D.** With 95% probability we expect an increment in the mean number of plant species between 0.14% and 0.15% per 1 meter increase in the highest elevation of an island.

*Assign your answers to the object `answer1.8`. Your answers Your answer should be one of "A", "B", "C", or "D", surrounded by quotes.*

In [ ]:
# answer1.8 <- 

### BEGIN SOLUTION
answer1.8 <- "B"
### END SOLUTION

In [ ]:
test_1.8()

The dataset contains other continuous covariates that may be useful in understanding the number of species that are growing in each island. 

This time, we'll estimate a Poisson regression using all the covariates available.  

**Question 1.9**
<br>{points: 1}

Using `glm()`, estimate a Poisson regression model with `species` as a response and all other continuous covariate.

> note in the skeleton provided how to exclude a small number of columns

Call the model `species_poisson_model`.
    
*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*    

In [ ]:
# species_poisson_model <- ...(...~ . - island,
#                            ...,
#                            ...)

### BEGIN SOLUTION
species_poisson_model <- glm(formula = species ~ . - island,
                           data = galapagos,
                           family = poisson)
### END SOLUTION

summary(species_poisson_model)

In [ ]:
test_1.9()

**Question 1.10**
<br>{points: 1}

Use `exponentiate = TRUE` in `tidy()` to get the estimates of the coefficients for the model of the mean number of plant species (i.e., $E[\texttt{species}_i|\texttt{elevation}_i]$). Include the corresponding 95% confidence intervals.

Round numeric results to 4 decimal places.

Call the resulting object `species_poisson_exp_results`. 

*Again, write your code without a skeleton in the cell below, and run it.*

In [ ]:
# write your own code here 

### BEGIN SOLUTION
species_poisson_exp_results <- 
    species_poisson_model %>%
    tidy(exponentiate = TRUE, conf.int = TRUE) %>%
    mutate_if(is.numeric, round, 4)
### END SOLUTION

species_poisson_exp_results

In [ ]:
test_1.10()

**Question 1.11**
<br>{points: 1}

Would the interpretation of the coefficient for `elevation` change with the new model? if so, provide a new interpretation, otherwise, re-write it your previous one.

Again provide an interpretation for the model of the mean of the number of plant species. 

> *Your answer goes here.*

### BEGIN SOLUTION
Keeping all other variables in the model constant, at any value, an increase of 1 meter in the maximum elevation of an island is associated with a change in the mean (or average) number of plant species found by a factor of $1.0035$, i.e., an increase of $0.35\%$.

Rubric note: either interpretation is correct

(-.5): if they don't say "keeping all variables constant" 

(-.5): if they provide an interpretation in terms of the log mean

(-1): if the raw estimates are used with a multiplicative interpretation 

(-1): if there's no adjustment
### END SOLUTION

**Question 1.12**
<br>{points: 1}

Santa Cruz is the home to the largest human population in the Islands. Being close to this island can affect the growth of plant species.

The variable `scruz` measures the distance from each island to Santa Cruz (in kilometers). 

Based on the results stored in `species_poisson_exp_results`, which of the following claims is(are) true?

**A**: Keeping all other variables in the model constant, at any value, an increase of 1 kilometers in the proximity to Santa Cruz is associated with a change in the mean number of plant species by a factor of $0.9943$.

**B**: Keeping all other variables in the model constant, at any value, an increase of 1 kilometers in the proximity to Santa Cruz is associated with a $0.57\%$ decrease in the mean number of plant species.

**C**: Keeping all other variables in the model constant, at any value, an increase of 5 kilometers in the proximity to Santa Cruz is associated with a $2.8\%$ decrease in the mean number of plant species since $(0.9943^{5} -1)\times 100\% = - 2.8\%$.

**D**: Keeping all other variables in the model constant, at any value, an increase of 1 kilometers in the proximity to Santa Cruz is associated with a increase in the mean number of plant species by a factor of $0.57\%$.

*Assign your answer to the object `answer1.12`. Your answers have to be included in a single string indicating the correct options **in alphabetical order** and surrounded by quotes (e.g., `"AB"` indicates you are selecting the two options).*

In [ ]:
# answer1.12 <- 

### BEGIN SOLUTION
answer1.12 <- "ABC"
### END SOLUTION

In [ ]:
test_1.12()

### Residuals

For a Poisson random variable, the variance equals the mean. Thus, the residuals are usually adjusted by the standard deviation of each observation. 

For the case of Poisson regression, the Pearson residuals are defined as:

$$\frac{y_i - \hat{\lambda}_i}{\sqrt{\hat{\lambda}_i}}$$

where $\hat{\lambda}_i$ is the fitted value using the exponentiated model, in R the `type = "response"`

**Question 1.13**
<br>{points: 1}

Use the function `augment()` to add fitted values and residuals to the `galapagos` dataset. Select only the columns `species`, `.fitted` and `.resid`.

Next, add to the tibble the residuals computed with the function `residuals` on the estimated model `species_poisson_model`, using `type = "response"` and `type = "pearson"`.

Compute the Pearson residuals using the fitted values and add them to the tibble.

*Assign your answer to the object `species_poisson_residuals`. Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# species_poisson_residuals <- ... %>%
#               ...() %>%
#               dplyr::select(...,..., ...) %>%
#               mutate(pred_mean = ...,
#                      resid_raw = residuals(..., type = "response"),
#                      resid_pearson =...(...,...),
#                      pearson_byhand = .../sqrt(...))

### BEGIN SOLUTION
species_poisson_residuals <- species_poisson_model %>%
              augment() %>%
               dplyr::select(species,.fitted, .resid) %>%
                     mutate(pred_mean = species_poisson_model$fitted,
                     resid_raw = residuals(species_poisson_model, type = "response"),
                     resid_pearson =residuals(species_poisson_model,"pearson"),
                     pearson_byhand = (species-pred_mean)/sqrt(pred_mean))
             
### END SOLUTION

head(species_poisson_residuals)

In [ ]:
test_1.13()

**Question 1.14**
<br>{points: 1}

Based on the results obtained in `species_poisson_residuals`, which of the following observations are correct?

**A.** The column `.resid` in the output of `augment()` contains the values of the response minus the predicted means of number of plants.

**B.** The `.resid` computed by `augment()` are neither the raw or the Pearson residuals.

**C.** The Pearson's residuals can be computed plugging the `species` and `.fitted` in the formula given in the previous cell.

**D** The `.fitted` values can be interpreted as the predicted mean of plant species for the islands in our sample.

*Assign your answers to the object answer1.14. Your answers must be included in a single string indicating the correct options in alphabetical order and surrounded by quotes (e.g., "ABC" indicates you are selecting the first three options).*

In [ ]:
# answer1.14 <- 

### BEGIN SOLUTION
answer1.14 <- "B"
### END SOLUTION

In [ ]:
test_1.14()

### Overdispersion

As in Logistic regression, a problem with Poisson regression is that the variance of a Poisson random variable equals the mean, which in practice may not be observed.

Thus, Poisson regression usually exhibits overdispersion. An easy way to check is to fit a model using the *quasilikelihood* method and check if the dispersion parameter is very different from 1.

In [ ]:
species_poisson_quasi <- summary(glm(
    formula = species ~ . -island,
    data = galapagos,
    family = quasipoisson))

species_poisson_quasi

While there are formal tests and other function to estimate overdispersion, there is a clear evidence of overdispersion here (31.75 is much larger than 1.

Overdispersion affects the standard errors and thus the inference results of the maximum likelihood estimated model. Thus, in this case the quasilikelihood estimates provide more reliable results.

### You are done!!